# Overview
We investigate the use case of __whistleblowing__, where someone wants to announce a message to the 'wider world', without having to reveal their identity. 

More specifically, we have a quantum network of $n$ nodes that we also refer to as __clients__, and one special node (Alice), want to announce a message $M$ of $m$ bits long. Alice uses __anonymous broadcasting__ to transmit her message, which is a functionality that can be implemented in quantum networks with various protocols.

## Network architecture and server assumptions
We assume that the $n$ clients are all connected to a central server $S$, and that there are no quantum links between the clients themselves. 
For now, we assume that we have a symmetric network, i.e.~the noise levels will be the same over all links. This will allows us to extrapolate the rates (see [Figures of Merit](#figures-of-merit)) from calculating only one link, most notably for the bi-partite case (see [Bi-partite protocol](#bi-partite-protocol))

We investigate two main settings:
- The server and the quantum links are __trusted__, which means that the nodes obtain the states from the server that they expect (up to noise), and do not need to perform verification.
- The server and the quantum links are __untrusted__, which means that the clients might not obtain the state that they expect, or that their state is actually part of larger pure state together with an eavesdropper. This means that they have to __verify__ their states.

## Figures of merit
In our simulations, we are interested in two main things:
- The bit rate, i.e.~the number of bits that Alice can anonymously announce per second. Here, higher number of bits per second is better.
- The time to announce, i.e.~the time it takes to announce a message of a fixed length. Here, lower number of seconds is better.

In general, these rates will depend on the number of rounds/states that are being used. Like with QKD, finite key effects arise, especially when we have a small number of rounds.

These rates can be computed without having to perform the 'full' simulation, where the clients actually perform the decoding etc. Therefore, we first focus on calculating rates, instead of simulating the final post-processing steps: those are classical, and not really interesting for us at the moment.

## Different protocols: Bi-partite vs. multi-partite entanglement
We have two different protocols that we can use for anonymous transmission:
- The first protocol is essentially classical, but uses bi-partite private communication between all $n$ clients. These private channels are provided by QKD, utilising bi-partite entanglement.
- The second protocol is a quantum protocol, that uses GHZ states shared between all clients. The non-local correlations of the GHZ state are used to transmit the message. It needs, in principle, fewer network uses than the bi-partite approach.

### Bi-partite protocol
The bi-partite protocol is based on the notification protocol of [Broadbent & Tapp](https://arxiv.org/abs/0706.2010), but adapted to only transmit a bit to everyone. Essentially, it has two rounds of communication to transmit a single bit; in each round every client sends a bit to every other client over a private channel. Hence, every client needs to share $2$ secret bits with every other client, for which they use QKD. There are $\frac{n\left(n-1\right)}{2}$ such combinations.

An important point is what the distributing server looks like:
- If the distributing server can only prepare one Bell pair at a time and send it to two nodes, there are $2\frac{n\left(n-1\right)}{2} = n\left(n-1\right)$ rounds necessary to establish all the necessary secret bits.
- If the server can create multiple entangled pairs at the same time and share them between separate pairs of clients in parallel, there are fewer rounds necessary. If there are an even number of nodes, there are $2*(n-1)$ rounds necessary (first round client 1 with 2, second round client 1 with 3 etc). If there are an odd number of nodes, each time one is left out, so there are $2*n$ rounds necessary.
- The server can create multiple entangled pairs, but the quality of the distributed Bell pairs deteriorates in doing so. In this case, the 'base' pairwise rate needs to reflect this.

Because we have the assumption that everything is symmetric in the network, to calculate the rates for the entire protocol, we can just calculate the rates for one single link (e.g. client 1 and client 2), and then adjust that rate according to how many concurrent network uses there should be.

If the server is trusted, the clients are sure that they share Bell pairs (up to noise), and therefore do not have to perform any privacy amplification. They need to perform error correction on their generated bits, which will reduce the bit rate, but (in an idealized setting that we use) this effect won't be dependent on the block size. We can assume, as is customary, that the ($Z$-basis) error rate can be pre-determined, so that we don't need to give up a portion for parameter estimation.

If the server is not trusted, the clients need to perform verification/parameter estimation to estimate their $X$-basis error rates. This gives up a portion of the raw key, so this will affect the bitrate. Here finite key effects play a role, because the estimation of the $X$-basis error rate suffers from statistical uncertainty.

### Multi-partite protocol
The multi-partite protocol is based on the anonymous transmission protocol of [Christandl & Wehner](https://arxiv.org/abs/quant-ph/0409201), which allows Alice to anonymously transmit one bit of message using a GHZ state. 

#### Basic protocol
During the protocol, every client measures in their qubit in the $X$ basis, obtaining outcome $o_{i}$. Alice then changes her outcome depending on her message: $o_{i} \rightarrow o_{i} \oplus m$.

Then, every client announces their outcome $o_{i}$.

For GHZ states i then holds that $\bigoplus_{i} o_{i} = m$ (i.e. all outcomes together are correlated), while for every strict subset $A$ of the network it holds that $\{o_{i}\}_{i \in A}$ is a set of uniformly random and uncorrelated bits. So, every client can calculate $b$, but Alice's announcement of $o_{i} \oplus m$ is completely indistinguishable from thother announcements, so she remains anonymous.

Due to noise, the calculated parity will not always be exacrtly $m$, but will be faulty. Therefore, Alice encodes her message into an error correcting code and repeated steps are necessary. The error rate of the parities (either determined using parameter estimation, or just assumed as pre-knowledge) will determine the coding rate, so the anonymous transmission bit rate will depend on this error rate.

#### Untrusted server
The above works and is anonymous if the server is trusted, so that the clients indeed share a GHZ state. If the server is untrusted, it could share any other state instead. Consider the case where the server shares a set of $X$-basis eigenstates (random eigenstates for every client, but the server keeps track of which ones she shared).

Every client measures in the $X$ basis, so the server knows what measurement outcomes they should get, and therefore what announcements they should make. Alice, encoding her bit into her outcome, announces the wrong outcome if her bit message was $m = 1$, and will therefore give herself up as Alice to the server.

To combat this, the clients need to verify that they have indeed a GHZ state. They can do this by performing measurements in different bases, so that if those measurement outcomes are correct, the state 'has' to be a GHZ state. Indeed, the GHZ state is a stabilizer state, giving fixed corelations in certain measurement bases. 

We already know that if all clients measure in the $X$ basis, the parity of the outcomes is fixed. If an even number of clients measure in the $Y$ basis instead, the parity will also be fixed. Moreover, no other state can have this: states that give the correct parity for the $X$-basis measurements that are different from the GHZ state, will give the wrong parities in at least one other of these measurement settings.

However, the clients cannot arrange/coordinate the measurement settings, because the server will known this (especially when some clients are untrusted, colluding with the server). 
To still have a working scheme, all clients can measure randomly in the $X$ or $Y$ basis. That means that on average $50$% of the rounds will have an even number of $Y$-basis measurements. 

After every client has finished their measurements (and shared that fact with the network), they can ask a public source of randomness to pick a subset of the rounds. These rounds can then be used to perform verification of the measurement results __without__ Allice encoding her message. It is important that the measurement outcomes are announced first, and that the measurement results are only announced afterwards. This means that a dishonest node cannot adapt their outcome before announcing so that 'artificially' the correct parity is found (and Alice is tricked into thinking she can safely announce her encoded bit).

There are also other ideas to select these verification rounds.
Instead of announcing all measurement outcomes from these rounds straight away, the clients could also use a PARITY subprotocol to calculate the parity of the rounds. (Check this:) this limits the possibility of dishonest clients cheating.

Alternative method would be to measure $Z$-basis correlations, and then use entropic uncertainty relations to determine that X-based measurements cannot be known/correlated with Eve. But this is just an idea, and much more work would be needed to realise this.

## Overview of different settings
| Syntax                                    | Bi-partite                            | Multi-partite                                     |
| -----------                               | -----------                           | ------------                                      |
| Trusted server                            | Easiest (only E.C.)                   | Easy-ish (GHZ states + E.C.)                      |
| Untrusted server                          | Easy (standard QKD: E.C. and P.A.)    | Hard (GHZ states, E.C. and Verification scheme)   |
| Untrusted server  & non-symmetric noise   | Easy (standard QKD: E.C. and P.A.)    | Hardest (GHZ states, E.C. and Verification scheme, noise channel estimation scheme)   |